![](https://drive.google.com/uc?export=view&id=1ZjLiGU2WX6NveSD0tsTpZwUA-FYC7lnI)

# Running the baseline and evaluation for CAD1 Task2

This tutorial walks through the process of running the CAD1 Task2 baseline using the shell Interface



The tools and recipes included in the <a href = 'https://github.com/claritychallenge/clarity'>clarity challenge repository</a> have been designed so that they can be integrated into python scripts that entrants may use in order to generate custom datasets or to expand the default datasets using new audio or varying data creation parameters. However, for convenience, the baseline clarity tools can be accessed in the command line interface (CLI) via shell scripts. 

The python and shell scripts included in the repository make use of <a href='https://hydra.cc/'>Hydra</a> and <a href='https://hydra.cc/docs/plugins/submitit_launcher/'>Submitit</a>, two technologies which streamline the configuration and parallel operation of python code on both local and high performnce computing (HPC) environments.

The use of hydra for configuration allows for the existing shell scripts to be easily redirected to include new audio data and modify the various parameters of the recipe.

---

## Setting the Location of the Project

For convenience, we are setting an environment variable with the location of the root working directory of the project. This variable will be used in various places throughout the tutorial. Please change this value to reflect where you have installed this notebook on your system. 

In [ ]:
%env NBOOKROOT=/content

env: NBOOKROOT=/content


## Cloning the Clarity Repository
We first need to install the Clarity package.

In [ ]:
print("Cloning git repo...")
!git clone --quiet https://github.com/claritychallenge/clarity.git
%cd clarity
%pip install -e .
import os
import sys

sys.path.append(f'{os.getenv("NBOOKROOT")}/clarity')
from IPython.display import clear_output

clear_output()
print("Repository installed")

Repository installed


## Get the demo data

We will be using music audio and listener metadata.

In [ ]:
import gdown

!gdown 1OWn0w39t_aeOOh514wRUgTw5uSwsoB17
!mv cadenza_data_demo.tar.xz recipes/cad1/task2/baseline
!tar -xvf cadenza_task2_data_demo.tar.xz

clear_output()
print("Data installed")

Data installed


## Changing working Directory

Next, we change working directory to the location of the shell scripts we wish to run.

In [ ]:
%cd {os.environ['NBOOKROOT']}/clarity/recipes/cad1/task2/baseline

/content/clarity/recipes/cad1/task2/baseline


## Inspecting Existing Configuration

All of the included shell scripts take configurable variables from the yaml files in the same directory as the shell script.Typically these are named <code>config.yaml</code>, however, other names may be used if more than one shell script is in a directory.

We can inspect the contents of the config file using <code>!cat</code>:

In [ ]:
!cat config.yaml

path:
  root: ../../cadenza_task2_data_demo/cad1/task2
  audio_dir: ${path.root}/audio
  metadata_dir: ${path.root}/metadata
  music_dir: ${path.audio_dir}/music
  hrtf_dir: ${path.audio_dir}/eBrird
  listeners_train_file: ${path.metadata_dir}/listeners.train.json
  listeners_valid_file: ${path.metadata_dir}/listeners.valid.json
  scenes_file: ${path.metadata_dir}/scenes.json
  scenes_listeners_file: ${path.metadata_dir}/scenes_listeners.json
  hrtf_file: ${path.metadata_dir}/eBrird_BRIR.json
  exp_folder: ./exp # folder to store enhanced signals and final results

sample_rate: 44100

nalr:
  nfir: 220
  fs: ${sample_rate}

compressor:
  threshold: 0.7
  attenuation: 0.1
  attack: 5
  release: 20
  rms_buffer_size: 0.064

soft_clip: False

enhance:
  average_level: -14   # Average level according Spotify's levels
  min_level: -19

evaluate:
  set_random_seed: True
  small_test: False
  save_intermediate_wavs: False
  split: valid # train, valid
  batch_size: 1   # Number of batches
  b

The general organisation of the config files is hierarchical, with property labels depending on the script in question. The config file for the enhance and evaluate recipes contains configurable paramaters for both scripts. These include:
- Paths for the locations of audio files, metadata and the export location for generated files
- Sample rate use in the recipe
- Paramaters for the NAL-R fitting
- Paramaters for the automatic gain control (AGC) compressor used in the baseline enhancer
- Parameters for the challenge evaluator
- Parameters necessary for Hydra to run




The path.root parameter defaults to the root of the baseline and must be overrided with the dataset root path when the python script is called in the command line.

e.g

```
user:~$ python mypythonscript.py path.root='/path/to/data/set' 
```

In this notebook we will use the environment variable <code>$NBOOKROOT</code> which we defined at the start of the tutorial.

Note the lack of slash at the end of the <code>path.root</code> argument string. If you inspect a variable such as <code>path.metadata_dir</code> you will see that this slash is already included in the line.

```
path:
  root: ./
  metadata_dir: ${path.root}/metadata

```

The general form for overriding a parameter in the CLI is dot indexed. For the following entry in a <code>config.yaml</code> file:
```
A:
  B:
    parameter_0: some_value
    parameter_1: some_other_value
```
The CLI syntax to override those values would be:

```
User:~$ python myscript.py A.B.parameter_0="new_value" A.B.parameter_1="another_new_value"
```

## Shell Scripts 

Typically, as stated above, all the work is done within python with configurable variables supplied by a <code>yaml</code> file which is parsed by Hydra inside the python code. 

The execution of this code is performed in the CLI and new configuration variable values are supplied as arguments to override defaults. 

---
### Additional steps for Colab Notebooks
This version of this tutorial is designed to run on Google Colab. The editable installation of the clarity repository is by default not visible to the python interpreter in this environment, even though the installation cell above makes the clarity tools visible to the iPython interpreter. 

As such, we need to make sure that the standard python interpreter called in the shell magic that follows below has the location of the clarity packages in the PYTHONPATH variable.

For local environments, this step may not be necessary.

In [ ]:
%env PYTHONPATH=$PYTHONPATH:/content/clarity

env: PYTHONPATH=$PYTHONPATH:/content/clarity


In [ ]:
%%shell
python enhance.py \
path.root=../cad1/task2

100% 20/20 [00:09<00:00,  2.13it/s]


In [ ]:
import pandas as pd
import IPython.display as ipd

In [ ]:
def audio_player_list(signals, rates, width=270, height=40, columns=None, column_align='center'):
    """Generate a list of HTML audio players tags for a given list of audio signals.

    Notebook: B/B_PythonAudio.ipynb

    Args:
        signals (list): List of audio signals
        rates (list): List of sample rates
        width (int): Width of player (either number or list) (Default value = 270)
        height (int): Height of player (either number or list) (Default value = 40)
        columns (list): Column headings (Default value = None)
        column_align (str): Left, center, right (Default value = 'center')
    """
    pd.set_option('display.max_colwidth', None)

    if isinstance(width, int):
        width = [width] * len(signals)
    if isinstance(height, int):
        height = [height] * len(signals)

    audio_list = []
    for cur_x, cur_Fs, cur_width, cur_height in zip(signals, rates, width, height):
        audio_html = ipd.Audio(data=cur_x.T, rate=cur_Fs)._repr_html_()
        audio_html = audio_html.replace('\n', '').strip()
        audio_html = audio_html.replace('<audio ', f'<audio style="width: {cur_width}px; height: {cur_height}px" ')
        audio_list.append([audio_html])

    df = pd.DataFrame(audio_list, index=columns).T
    table_html = df.to_html(escape=False, index=False, header=bool(columns))
    table_html = table_html.replace('<th>', f'<th style="text-align: {column_align}">')
    ipd.display(ipd.HTML(table_html))

Now we have the enhanced output. Below, we can load and play the audio to listen to examples of the results.

In [ ]:
!pip install more_itertools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from os import listdir
from os.path import isfile, join
from pathlib import Path
from scipy.io import wavfile
from more_itertools import windowed

import IPython.display as ipd

audio_path = Path("exp/enhanced_signals")
audio_files = [f for f in audio_path.glob('*/*/*') if f.suffix == '.wav'][:4]

signals = []
rates = []
columns = []
for file_to_play in audio_files:
  sample_rate, signal = wavfile.read(file_to_play)
  signals.append(signal)
  rates.append(sample_rate)
  columns.append(file_to_play.name)
for s, r, c in zip(
      windowed(signals, 2, step=2),
      windowed(rates, 2, step=2),
      windowed(columns, 2, step=2)): 
  audio_player_list(s, r, columns=c)

Output hidden; open in https://colab.research.google.com to view.

Now that we have enhanced audios we can use the evaluate recipe to generate HAAQI scores for the signals. The evaluation is run in the same manner as the enhancement script.

In [ ]:
%%shell
python evaluate.py \
path.root=../cad1/task2 

In [ ]:
from os import listdir
from os.path import isfile, join
from pathlib import Path
from scipy.io import wavfile
from more_itertools import windowed

import IPython.display as ipd

audio_path = Path("exp/evaluation_signals")
audio_files = [f for f in audio_path.glob('*/*/*') if f.suffix == '.wav'][:4]

signals = []
rates = []
columns = []
for file_to_play in audio_files:
  sample_rate, signal = wavfile.read(file_to_play)
  signals.append(signal)

  _, signal = wavfile.read(file_to_play.parent / "ref_signal_for_eval.wav")
  signals.append(signal)

  rates += [sample_rate, sample_rate]

  columns += [" - ".join(file_to_play.parts[-3:-1]) + "\n Evaluation Signal", "\n Reference Signal"]

for s, r, c in zip(
      windowed(signals, 2, step=2),
      windowed(rates, 2, step=2),
      windowed(columns, 2, step=2)): 
  audio_player_list(s, r, columns=c)

Output hidden; open in https://colab.research.google.com to view.

We hope that this tutorial has been useful and has explained the process for using the recipe scripts using the Hydra configuration system. This approach can be applied to all of the recipes that are included in the repository.